In [11]:
import pandas as pd
import pickle
from ast import literal_eval
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, cohen_kappa_score, f1_score , confusion_matrix
from nltk.corpus import stopwords
from sklearn.svm import LinearSVC , NuSVC

In [12]:
from bpemb import BPEmb
bpemb_de = BPEmb(lang="de", vs=25000, dim=300)

def bpemb_tokenize(sentence):
    return [i.replace("▁","") for i in bpemb_de.encode(sentence)]


In [13]:
# mlb =  pickle.load(open("mlb_author_year_without_num.pickle",'rb'))
# year_lb = pickle.load(open("lb_author_year_without_num.pickle",'rb'))
# vectorizer =  pickle.load(open("vectorizer_author_year_without_num.pickle",'rb'))

# X_train = pickle.load(open("X_train_author_year_without_num.pickle", "rb"))
# X_test = pickle.load(open("X_test_author_year_without_num.pickle", "rb"))
# y_train = pickle.load( open("y_train_author_year_without_num.pickle", "rb"))
# y_test = pickle.load(open("y_test_author_year_without_num.pickle", "rb"))
# le_mapping = pickle.load(open("le_author_year_mapping.pickle", "rb")) # LabelEncoder Mapping

In [15]:
vectorizer =  pickle.load(open("cp_vpt_vectorizer_without_num.pickle",'rb'))
X_train = pickle.load( open("cp_vpt_X_train_without_num.pickle", "rb"))
X_test = pickle.load(open("cp_vpt_X_test_without_num.pickle", "rb"))
y_train = pickle.load( open("cp_vpt_y_train_without_num.pickle", "rb"))
y_test = pickle.load(open("cp_vpt_y_test_without_num.pickle", "rb"))

In [16]:
import neptune

def build_model(clf, data):
    X_train, X_test, y_train, y_test = data
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred, y_test

def push_metrics(y_pred, y_true, tag,clf, project_qualified_name='raghavan/GermEval',
                api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5tbCIsImFwaV9rZXkiOiI4ZjhlOTM1OC0yY2UzLTRkMTgtOWVhNi04NzY2NTY3NzFhMDMifQ=='):
    neptune.init(api_token=api_token, project_qualified_name=project_qualified_name)
    neptune.create_experiment()
    neptune.append_tag(tag)
#     neptune.send_metric("parameters",clf.get_params())
    neptune.send_metric("accuracy", accuracy_score(y_true, y_pred))

    neptune.send_metric("precision", precision_score(y_true, y_pred, average='micro'))
    neptune.send_metric("recall", recall_score(y_true, y_pred, average='micro'))
    neptune.send_metric("kappa", cohen_kappa_score(y_true, y_pred))
    neptune.send_metric("f1_score", f1_score(y_true, y_pred, average='micro'))
    neptune.stop()

In [17]:
def build_model(clf, X_train,y_train):
    clf.fit(X_train, y_train)
    return clf

## Linear SVC

In [18]:
X_train.shape

(13301, 687116)

In [19]:
y_train.shape

(13301,)

In [21]:
X_test.shape

(3326, 687116)

In [23]:
y_test.shape

(3326,)

In [24]:
%%time
from scipy import sparse

X_full = sparse.vstack((X_train,X_test))
Y_full = np.concatenate((y_train,y_test))

CPU times: user 30.8 ms, sys: 49.3 ms, total: 80.2 ms
Wall time: 79.6 ms


In [29]:
print(X_full.shape)
print(len(Y_full))

(16627, 687116)
16627


In [30]:
%%time
lsvcclf = LinearSVC(random_state=42,verbose=1,max_iter=3000)
clf = build_model(lsvcclf,X_full,Y_full)

[LibLinear]CPU times: user 2.86 s, sys: 1.15 ms, total: 2.87 s
Wall time: 2.88 s


## Load the validation set and create predictions for it 

In [31]:
validation_data = pd.read_csv("../../data/labeled_validation_data.csv")

In [32]:
def combine_title_desc(row):
    """Combines the Title and Description fields in the given row
    and returns the combined result"""
    return str(row["title"]) + " " + str(row["description"])
    #return str(row["title"]) + " " + str(row["description"]) + " " + str(row["author"])

In [33]:
validation_tf_idf = vectorizer.transform(validation_data.apply(lambda row: combine_title_desc(row),axis=1))

In [15]:
# isbn_probas = pd.read_csv("predict_proba.csv")

In [16]:
# isbn_probas_averaged = isbn_probas.groupby("isbn").mean()

In [17]:
# isbn_probas_averaged.reset_index(inplace=True)

In [18]:
# isbn_probas_averaged.head(3)

,isbn,Architektur_&_Garten,Ganzheitliches_Bewusstsein,Glaube_&_Ethik,Kinderbuch_&_Jugendbuch,Künste,Literatur_&_Unterhaltung,Ratgeber,Sachbuch
0,4012390379198,0.096320,0.102583,0.112354,0.257898,0.102689,0.126343,0.099884,0.101929
1,4012390379235,0.095435,0.107051,0.111938,0.271743,0.100562,0.107367,0.101625,0.104281
2,4031812001752,0.101949,0.103545,0.099368,0.310302,0.107804,0.100911,0.082865,0.093256


In [19]:
# len(validation_data)

2079

In [20]:
# validation_data = validation_data.merge(isbn_probas_averaged,on="isbn",how="inner")

In [34]:
len(validation_data)

2079

In [35]:
from bpemb import BPEmb
bpemb_de = BPEmb(lang="de", vs=25000, dim=300)

def bpemb_tokenize(sentence):
    return [i.replace("▁","") for i in bpemb_de.encode(sentence)]


In [38]:
mlb =  pickle.load(open("ll_vpt_mlb_author._year_without_num.pickle",'rb'))
year_lb = pickle.load(open("ll_vpt_lb_author_year_without_num.pickle",'rb'))
vectorizer =  pickle.load(open("cp_vpt_vectorizer_without_num.pickle",'rb'))

# posterior_columns = isbn_probas.columns.tolist()

validation_data["authors"] = validation_data["author"].apply(lambda x:[ i.strip() for i in str(x).split(",")])

validation_data["published_date_parsed"] = pd.to_datetime(validation_data["published_date"],infer_datetime_format=True)

validation_data["year"] = validation_data["published_date_parsed"].apply(lambda x: x.year)

validation_tf_idf.shape

validation_data["authors"].values

pickle.dump(validation_data["authors"], open("validation_authors", "wb"))

from scipy import sparse

validation_author = mlb.transform(validation_data["authors"].values)
validation_year = year_lb.transform(validation_data["year"].values)
validation_all = sparse.hstack(\
    (validation_tf_idf,sparse.csr_matrix(validation_author)\
     ,sparse.csr_matrix(validation_year)\
#      ,sparse.csc_matrix(validation_data[posterior_columns].values.tolist())
    ))

In [39]:
validation_tf_idf.shape

(2079, 678426)

In [40]:
predictions = clf.predict(validation_all)

In [41]:
predictions

array([2, 3, 1, ..., 1, 1, 4])

In [42]:
pd.Series(predictions).to_csv("leaf_class_count_predictions_from_svc_validation_set.csv",index=False)

/home/evenuma/anaconda3/envs/deeplearning_36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


## Linear SVC

## Lets check the confusion matrix (Do not run this , this will take long time and will not yield good results)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
array = confusion_matrix(y_pred,y_test,labels=list((set(y_test))))
df_cm = pd.DataFrame(array,index=list((set(y_test))),columns=list((set(y_test))))
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True,fmt="0.5g")


In [14]:
set(y_test)

{1, 2, 3, 4}

In [31]:
from sklearn.metrics import classification_report

In [33]:
classification_report(predictions,y_test)

ValueError: Found input variables with inconsistent numbers of samples: [2079, 2910]

In [34]:
validation_data

,Unnamed: 0,title,description,categories,author,published_date,isbn,Architektur_&_Garten,Ganzheitliches_Bewusstsein,Glaube_&_Ethik,Kinderbuch_&_Jugendbuch,Künste,Literatur_&_Unterhaltung,Ratgeber,Sachbuch,authors,published_date_parsed,year
0,14548,Selbstständig werden im Glauben,Die Konfirmandenzeit wird für Jugendliche beso...,['Glaube & Ethik > Gemeindearbeit > Konfirmati...,Hans-Martin Lübking,2013-07-22,9783579074030,0.105653,0.098056,0.295717,0.111842,0.105847,0.087194,0.112128,0.083562,[Hans-Martin Lübking],2013-07-22,2013
1,14549,Das Buch der Schatten - Schwarze Seelen,Als Morgan von einer prophetischen Vision heim...,"['Kinderbuch & Jugendbuch > Echtes Leben, Real...",Cate Tiernan,2013-06-10,9783570380093,0.103372,0.102328,0.103462,0.275309,0.101941,0.107734,0.103773,0.102081,[Cate Tiernan],2013-06-10,2013
2,14550,Die Jesuitin von Lissabon,Antero Moreira de Mendonca hasst die Jesuiten....,['Literatur & Unterhaltung > Historische Romane'],Titus Müller,2018-11-12,9783453422650,0.103413,0.102265,0.108801,0.106406,0.098046,0.273474,0.118230,0.089365,[Titus Müller],2018-11-12,2018
3,14551,Die Essenz der Lehre Buddhas,Klar und verständlich führt der Dalai Lama in ...,['Glaube & Ethik > Spiritualität & Religion > ...,Dalai Lama,2014-02-10,9783453702479,0.102107,0.197191,0.182814,0.103979,0.098245,0.104443,0.105780,0.105441,[Dalai Lama],2014-02-10,2014
4,14552,Das letzte Kind,"Es ist ein Jahr vergangen, seit die 12-jährige...",['Literatur & Unterhaltung > Krimi & Thriller ...,John Hart,2010-08-13,9783641048198,0.101763,0.102709,0.090114,0.142231,0.097316,0.277360,0.102865,0.085642,[John Hart],2010-08-13,2010
5,14553,Die erste Nacht,Aus Keiras und Adrians Begegnung ist Liebe gew...,['Literatur & Unterhaltung > Frauenunterhaltun...,Marc Levy,2012-01-16,9783442376599,0.103115,0.105650,0.103296,0.092587,0.094618,0.312816,0.089295,0.098623,[Marc Levy],2012-01-16,2012
6,14554,Biokrieg,Bangkok in naher Zukunft: Klimawandel und die ...,['Literatur & Unterhaltung > Science Fiction >...,Paolo Bacigalupi,2011-02-08,9783453527577,0.104123,0.104074,0.103204,0.098204,0.103208,0.238356,0.112750,0.136081,[Paolo Bacigalupi],2011-02-08,2011
7,14555,Zartbitter ist das Glück,Fünf Freundinnen wagen auf Fidschi einen Neuan...,['Literatur & Unterhaltung > Romane & Erzählun...,Anne Østby,2017-08-28,9783336547913,0.103237,0.108764,0.107702,0.123190,0.106231,0.219025,0.115473,0.116377,[Anne Østby],2017-08-28,2017
8,14556,Zerstört,Eine Leiche in einem ausgebrannten Auto. Unter...,['Literatur & Unterhaltung > Krimi & Thriller ...,Karin Slaughter,2010-11-09,9783442372201,0.100496,0.108295,0.107494,0.095641,0.098817,0.272996,0.103150,0.113111,[Karin Slaughter],2010-11-09,2010
9,14557,Es war einmal ein Palästina,Eine meisterhafte Untersuchung des turbulenten...,"['Sachbuch > (Zeit-) Geschichte', 'Sachbuch > ...",Tom Segev,2006-07-17,9783570550090,0.104358,0.105049,0.105973,0.109041,0.088377,0.109292,0.104929,0.272982,[Tom Segev],2006-07-17,2006
